In [1]:
from transformers import ViltProcessor, ViltForQuestionAnswering
from PIL import Image

def vilt_load():
    
    model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa",device_map='auto')

    processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa",use_fast=True)
    
    n = model.num_parameters()/1000000000
    if(n<1):
        print(f'Base Model Paramters {n*1000:.0f}M')
    else:
        print(f'Base Model Paramters {n:.1f}B')
    return model,processor

def vilt_inference(img_path,question_text,model,processor):
    image = Image.open(img_path).convert('RGB')
    text = f'Answer the question in exactly one word:{question_text}'
    encoding = processor(image, text, return_tensors="pt").to(model.device)
    
    outputs = model(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()
    
    res = model.config.id2label[idx]

    return res

2025-05-01 10:34:37.877511: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746095678.101182      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746095678.166843      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model,processor = vilt_load()

config.json:   0%|          | 0.00/136k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/470M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/470M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Base Model Paramters 118M


In [3]:
img_path = '/kaggle/input/abo-small/images/small/00/00000529.jpg'
question_text = "How many wheels are present?"

print(vilt_inference(
            img_path=img_path,
            question_text=question_text,
            model=model,
            processor=processor
        ))

2


In [ ]:
from tqdm import tqdm
import pandas as pd

df = pd.read_csv('/kaggle/input/sqid-test-vqa/SQID_test_generated_vqa.csv')
# prepare a list to collect results
results = []

# iterate over each row, split into individual QA pairs, and predict
for _, row in tqdm(df.iterrows(), total=len(df), desc="Images"):
    img = row["image_path"]
    qs  = row["questions"].split("|")
    gts = row["answers"].split("|")

    for question, gt_answer in zip(qs, gts):

        pred = vilt_inference(img_path = img,
               question_text = question,
                model=model,
                processor=processor
            )

        # store
        results.append({
            "ground_truth":  gt_answer,
            "prediction":    pred
        })

# turn your list of dicts into a DataFrame
result_df = pd.DataFrame(results)

# save it
result_df.to_csv("VILT-Baseline.csv", index=False)

Images: 100%|██████████| 984/984 [03:30<00:00,  4.68it/s]


In [5]:
pd.read_csv('/kaggle/working/VILT-Baseline.csv')

,ground_truth,prediction
0,Five,4
1,Black,white
2,Rectangular,oval
3,Metal,metal
4,Glasses,glasses
...,...,...
4915,Tundra,yes
4916,Black,red
4917,Two,4
4918,Rectangular,no
